In [ ]:
pip install utils

In [ ]:
import logging
import os
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib inline

warnings.filterwarnings('ignore')
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [ ]:
!ls -lha ../input

In [ ]:
PLANET_KAGGLE_ROOT = os.path.abspath("../input/planets-dataset/planet/planet/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_classes.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [ ]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

In [ ]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [ ]:
label_onehot = labels_df.copy()

for label in label_list:
    label_onehot[label] = label_onehot['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
label_onehot.head()

In [ ]:
# Histogram of label instances
label_onehot[label_list].sum().sort_values().plot.bar()

In [ ]:
# Converting labels to lists to be used in the classification
labels_df['tags'] = labels_df['tags'].str.split(" ")
# Display head
labels_df.head()

In [ ]:
# Pull a list of 20000 image names
jpg_list = os.listdir(PLANET_KAGGLE_JPEG_DIR)[:20000]
# Select a random sample of 100 among those
np.random.shuffle(jpg_list)
jpg_list = jpg_list[:100]

print(jpg_list)

In [ ]:
labels_df.head()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

from datetime import datetime
from keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers

from utils import *

### Cleaning Dataset

In [ ]:
labels_df.isna().any()

### Splitting data to Train and Validation data

In [ ]:
labels_df.columns

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(labels_df['image_name'], 
    labels_df['tags'], 
    test_size=0.2, 
    random_state=20)
print("Number of posters for training: ", len(X_train))
print("Number of posters for validation: ", len(X_val))

In [ ]:
X_train = [os.path.join(PLANET_KAGGLE_JPEG_DIR, str(f)+'.jpg') for f in X_train]
X_val = [os.path.join(PLANET_KAGGLE_JPEG_DIR, str(f)+'.jpg') for f in X_val]
X_train[:3]

In [ ]:
y_train = list(y_train)
y_val = list(y_val)
y_train[:3]

### Visualizing the images with their labels

In [ ]:
fig = plt.figure()
fig.set_size_inches(25, 5)

for i in range(5):
    file_path = os.path.join(PLANET_KAGGLE_JPEG_DIR, jpg_list[i])
    title = labels_df[labels_df['image_name'] == jpg_list[i][:-4]].tags.values[0]
    image = Image.open(file_path)
    
    a = fig.add_subplot(1, 5, i+1)
    
    plt.imshow(image)
    a.set_title(title)
    

In [ ]:
nobs = 8 # Maximum number of images to display
ncols = 4 # Number of columns in display
nrows = nobs//ncols # Number of rows in display

style.use("default")
plt.figure(figsize=(18,4*nrows))
for i in range(nrows*ncols):
    ax = plt.subplot(nrows, ncols, i+1)
    plt.imshow(Image.open(X_train[i]))
    plt.title(y_train[i], size=10)
    plt.axis('off')

### Preprocessing

In [ ]:
print("Labels:")
mlb = MultiLabelBinarizer()

mlb.fit(y_train)

N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

In [ ]:
# transform the targets of the training and test sets
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)

In [ ]:
# Print example of movie posters and their binary targets
for i in range(3):
    print(X_train[i], y_train_bin[i])

### Dataset Creation

In [ ]:
IMG_SIZE = 224
CHANNELS = 3

In [ ]:
def parse_function(filename, label):
    
   # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [ ]:
BATCH_SIZE = 100

SHUFFLE_BUFFER_SIZE = 512

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def create_dataset(filenames, labels, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    
    if is_training == True:
        dataset = dataset.cache()
        
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
train_ds = create_dataset(X_train, y_train_bin)
val_ds = create_dataset(X_val, y_val_bin)

In [ ]:
for f, l in train_ds.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

In [ ]:
for image_batch, labels_batch in train_ds.take(1):
    print(image_batch.shape)
    print(labels_batch.shape)
    

### Model Building

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(100, activation='relu', name='hidden_layer'),
    layers.Dropout(0.3),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(N_LABELS, activation='sigmoid', name='output')
])

model.summary()

In [ ]:
for batch in train_ds:
    print(model.predict(batch)[:1])
    break

### Model Training

In [ ]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [ ]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
LR = 0.01
EPOCHS = 20

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=macro_soft_f1,
    metrics=[macro_f1]
)

In [ ]:
start = datetime.now()
history = model.fit(train_ds,
                   epochs=EPOCHS,
                   validation_data=create_dataset(X_val, y_val_bin))
print('\nTraining took {}'.format(print_time(datetime.now()-start)))